In [ ]:
# import simulation stuff
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

In [ ]:
# import models and solvers
# from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
# from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
# from wholebody_croccodyl_solver import WholeBodyDDPSolver
# from wholebody_croccodyl_model import WholeBodyModel
# import numpy as np

# # DDP warm-start
# wbd_model = WholeBodyModel(conf)
# ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
# ddp_planner.solve()
# ddp_sol = ddp_planner.get_solution_trajectories()
# centroidal_warmstart = ddp_sol['centroidal']
# q_warmstart = ddp_sol['jointPos']
# qdot_warmstart = ddp_sol['jointVel']
# x_warmstart = []
# u_warmstart = []
# rmodel, rdata = conf.rmodel, conf.rdata
# for k in range(len(centroidal_warmstart)):
#     x_warmstart.append(
#         np.concatenate(
#             [centroidal_warmstart[k],
#              q_warmstart[k], 
#              qdot_warmstart[k]]
#             )
#         )
#     u_warmstart.append(np.concatenate([np.zeros(30)]))
# # nominal traj-opt
# model_nom = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
# solver_nom = CentroidalPlusLegKinematicsAcadosSolver(
#     model_nom, x_warmstart, u_warmstart, MPC=True)
# x_nom, u_nom = solver_nom.solve()

In [ ]:
# import pinocchio as pin
# import utils

# # visualize motion in meshcat
# dt = conf.dt
# dt_ctrl = 0.01
# N_ctrl =  int(dt/dt_ctrl)
# # initialize end-effector trajectories
# FL_nom = np.zeros((3, x_nom.shape[0])).astype(np.float32)
# FR_nom = np.zeros((3, x_nom.shape[0])).astype(np.float32)
# HL_nom = np.zeros((3, x_nom.shape[0])).astype(np.float32)
# HR_nom = np.zeros((3, x_nom.shape[0])).astype(np.float32)

# if conf.WITH_MESHCAT_DISPLAY:
#     viz = pin.visualize.MeshcatVisualizer(
#     robot.model, robot.collision_model, robot.visual_model)
#     try:
#         viz.initViewer(open=True)
#     except ImportError as err:
#         print(err)
#         sys.exit(0)
#     viz.loadViewerModel()
# # add nominal contact surfaces
# s = conf.step_adjustment_bound
# for i, contacts in enumerate(conf.contact_sequence):
#     for contact_idx, contact in enumerate(contacts):
#         if contact.ACTIVE:
#             t = contact.pose.translation
#             if contact.CONTACT == 'FR' or contact.CONTACT == 'FL':
#                 utils.addViewerBox(
#                     viz, 'world/contact'+str(i)+str(contact_idx), 
#                     s, s, 0., [1., .2, .2, .5]
#                     )
#             if contact.CONTACT == 'HR' or contact.CONTACT == 'HL':
#                 utils.addViewerBox(
#                     viz, 'world/contact'+str(i)+str(contact_idx),
#                      s, s, 0., [.2, .2, 1., .5]
#                     )       
#             utils.applyViewerConfiguration(
#                 viz, 'world/contact'+str(i)+str(contact_idx), 
#                 [t[0], t[1], 0., 1, 0, 0, 0]
#                 )

# # visualize nominal motion
# for k in range(conf.N):
#     q_base_next = np.array(
#             model_nom.casadi_model.q_plus(
#                 x_warmstart[k][3:7], x_nom[k, 12:15]
#                 )
#             ).squeeze()
#     q = np.concatenate(
#         [x_nom[k, 9:12], 
#         q_base_next,
#             x_nom[k, 15:27]]
#         )
#     pin.framesForwardKinematics(rmodel, rdata, q)
#     FL_nom[:, k] = rdata.oMf[rmodel.getFrameId('FL_FOOT')].translation
#     FR_nom[:, k] = rdata.oMf[rmodel.getFrameId('FR_FOOT')].translation
#     HL_nom[:, k] = rdata.oMf[rmodel.getFrameId('HL_FOOT')].translation
#     HR_nom[:, k] = rdata.oMf[rmodel.getFrameId('HR_FOOT')].translation            
#     for j in range(12): 
#         viz.display(q)

In [ ]:
#import models and solvers
from centroidal_plus_double_integrator_kinematics_acados_solver import CentroidalPlusLegKinematicsAcadosSolver
from centroidal_plus_double_integrator_kinematics_casadi_model import CentroidalPlusLegKinematicsCasadiModel
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
from utils import interpolate_one_step
import pinocchio as pin
import numpy as np
import time 

# DDP warm-start
wbd_model = WholeBodyModel(conf)
ddp_planner = WholeBodyDDPSolver(wbd_model, MPC=False, WARM_START=False)
ddp_planner.solve()
ddp_sol = ddp_planner.get_solution_trajectories()
centroidal_warmstart = ddp_sol['centroidal']
q_warmstart = ddp_sol['jointPos']
qdot_warmstart = ddp_sol['jointVel']
x_warmstart = []
u_warmstart = []
rmodel = conf.rmodel 
rdata = rmodel.createData()
for k in range(len(centroidal_warmstart)):
    x_warmstart.append(
        np.concatenate(
            [centroidal_warmstart[k],
             q_warmstart[k], 
             qdot_warmstart[k]]
            )
        )
    u_warmstart.append(np.concatenate([np.zeros(30)]))
# construct model and solver
model = CentroidalPlusLegKinematicsCasadiModel(conf, STOCHASTIC_OCP=False)
solver = CentroidalPlusLegKinematicsAcadosSolver(
    model, x_warmstart, u_warmstart, MPC=True)
# dimensions
nx, nu = solver.nx, solver.nu
N_traj, N_mpc = solver.N_traj, solver.N_mpc
dt, dt_ctrl = conf.dt, conf.dt_ctrl
N_interpol = int(dt/dt_ctrl)
# trajectory contact references
contacts_logic = solver.contact_data['contacts_logic']
contacts_position = solver.contact_data['contacts_position']
contacts_norms = solver.contact_data['contacts_orient']
x_ref_mpc, u_ref_mpc = solver.x_ref_mpc, solver.u_ref_mpc
# get acados solver object
acados_solver = solver.acados_solver
# create open-loop tuples
solver.X_sim = np.zeros((N_traj, N_mpc+1, nx))
solver.U_sim = np.zeros((N_traj, N_mpc, nu))
# create closed-loop tuples
X_sol = np.zeros((N_traj+1, nx))
U_sol = np.zeros((N_traj, nu))
x0 = np.concatenate([solver.x_init[0][:9],
                     solver.x_init[0][9:12],
                     np.zeros(3),
                     solver.x_init[0][16:28],
                     solver.x_init[0][28:31], 
                     np.zeros(3), 
                     solver.x_init[0][34:]])
X_sol[0] = np.copy(x0)
STOCHASTIC = model._STOCHASTIC_OCP
# initial warm-start 
x_warm_start_N = np.copy(x_ref_mpc[:N_mpc])
u_warm_start_N = np.copy(u_ref_mpc[:N_mpc])
# get dynamics jacobians
A = model.casadi_model.Jx_fun
B = model.casadi_model.Ju_fun
# get forward kinematics and jacobians expressions
ee_fk = model.casadi_model.ee_fk
ee_jacobians = model.casadi_model.ee_jacobians
step_bound = solver.step_bound
# contact location constraint matrix dimensions
nb_lateral_contact_loc_constr = solver.nb_contacts*2
nb_vertical_contact_loc_constr = solver.nb_contacts
# joint PD gains
Kp = 2.5*np.eye(12)
Kd = 0.01*np.eye(12)
# 
# moving horizon loop
for traj_time_idx in range(N_traj):
    print('\n'+'=' * 50)
    print('MPC Iteration ' + str(traj_time_idx))
    print('-' * 50)
    # update initial conditon
    acados_solver.set(0, "lbx", x0)
    acados_solver.set(0, "ubx", x0)
    # get horizon references 
    horizon_range = range(traj_time_idx, traj_time_idx+N_mpc)
    x_ref_N = x_ref_mpc[horizon_range]
    u_ref_N = u_ref_mpc[horizon_range]
    contacts_logic_N = contacts_logic[horizon_range]
    contacts_norms_N = contacts_norms[horizon_range]
    contacts_position_N = contacts_position[horizon_range]
    Sigma_k = np.zeros((nx, nx))
    # OCP loop
    for mpc_time_idx in range(N_mpc):
        # state stage reference
        x_ref_k = np.concatenate([
            x_ref_N[mpc_time_idx][:9],
            x_ref_N[mpc_time_idx][9:12],
            np.zeros(3),
            x_ref_N[mpc_time_idx][16:28],
            x_ref_N[mpc_time_idx][28:31], 
            np.zeros(3), 
            x_ref_N[mpc_time_idx][34:]
            ])
        # control stage reference
        u_ref_k = u_ref_N[mpc_time_idx]                
        # parameters stage references
        contacts_logic_k = contacts_logic_N[mpc_time_idx]
        contacts_position_k = contacts_position_N[mpc_time_idx]
        contacts_norms_k = contacts_norms_N[mpc_time_idx].flatten()
        qref_base_k = x_ref_N[mpc_time_idx][12:16]
        params_k = np.concatenate([
            contacts_logic_k, 
            contacts_position_k, 
            contacts_norms_k, 
            qref_base_k
            ])
        # update paramters and tracking cost
        y_ref_k = np.concatenate([x_ref_k, u_ref_k])
        acados_solver.set(mpc_time_idx, 'p', params_k)
        acados_solver.cost_set(mpc_time_idx,'yref', y_ref_k)
        # warm-start stage nodes 
        if traj_time_idx == 0:
            x_warm_start_k = x_ref_k
            u_warm_start_k = u_ref_k
        else:
            x_warm_start_k = x_warm_start_N[mpc_time_idx]
            u_warm_start_k = u_warm_start_N[mpc_time_idx]    
        acados_solver.set(mpc_time_idx, 'x', x_warm_start_k)
        acados_solver.set(mpc_time_idx, 'u', u_warm_start_k)
        # propagate uncertainties 
        if STOCHASTIC:
            A_k = A(x_warm_start_k, u_warm_start_k, params_k)
            B_k = B(x_warm_start_k, u_warm_start_k,params_k)  
            K_k = solver.compute_riccatti_gains(A_k, B_k)
            Sigma_next = solver.propagate_covariance(A_k, B_k, K_k, Sigma_k) 
        # get the generalized position vector at the jth SQP iteration
        base_posj_k = np.array(x_warm_start_k[9:12])
        lambdaj_k = x_warm_start_k[12:15]
        joint_posj_k = np.array(x_warm_start_k[15:27])
        qj_k = np.concatenate(
            [base_posj_k, 
            np.array(
                model.casadi_model.q_plus(qref_base_k, lambdaj_k*solver.dt)
                ).squeeze(),
            joint_posj_k]
        )
        dqj_k = np.concatenate(
            [base_posj_k, 
            lambdaj_k,
            joint_posj_k]
            )
        # intialize linearized contact location constraint matrices
        C_lateral = np.zeros((nb_lateral_contact_loc_constr, nx))
        lg_lateral = np.zeros(nb_lateral_contact_loc_constr)
        ug_lateral = np.zeros(nb_lateral_contact_loc_constr)
        C_vertical = np.zeros((nb_vertical_contact_loc_constr, nx))
        lg_vertical = np.zeros(nb_vertical_contact_loc_constr)
        ug_vertical = np.zeros(nb_vertical_contact_loc_constr)
        # fill constraints for each end-effector
        for contact_idx in range(solver.nb_contacts):
            contact_position_param = \
                contacts_position_k[contact_idx*3:(contact_idx*3)+3] 
            ee_linear_jac = ee_jacobians[contact_idx](q=qj_k)['J'][:3, :]
            contact_logic = contacts_logic_k[contact_idx]               
            ee_Jlin_times_qj = ee_linear_jac @ dqj_k
            contact_fk = ee_fk[contact_idx](q=qj_k)['ee_pos']
            # activate contact location constraints only at the impact time, otherwise
            # contact location frame velocity is activated 
            # if traj_time_idx > 0 or not contacts_logic_N[mpc_time_idx-1][contact_idx]:
            #     print(traj_time_idx)
            # lateral part (x-y direction)
            for lateral_constraint_idx in range(2):
                idx = contact_idx*2 + lateral_constraint_idx
                constraint_row = ee_linear_jac[lateral_constraint_idx, :]
                C_lateral[idx, 9:27] = contact_logic*(constraint_row)
                temp = contact_position_param[lateral_constraint_idx] \
                        + ee_Jlin_times_qj[lateral_constraint_idx] \
                        - contact_fk[lateral_constraint_idx, :]
                # compute back-off mangnitude (only for lateral part)
                if STOCHASTIC:
                    backoff = solver.eta*np.sqrt(
                        (constraint_row @ Sigma_next[9:27, 9:27]) @ constraint_row.T 
                        )
                else:
                    backoff = 0.
                lg_lateral[idx] = contact_logic*(
                    temp - step_bound + backoff
                    )
                ug_lateral[idx] = contact_logic*(
                    temp + step_bound - backoff
                    )
            # vertical part (z-direction)
            C_vertical[contact_idx, 9:27] = contact_logic*(ee_linear_jac[2, :])
            temp = ee_Jlin_times_qj[2] - contact_fk[2] + contact_position_param[2]
            lg_vertical[contact_idx] = contact_logic*temp 
            ug_vertical[contact_idx] = contact_logic*temp
        # add contatenated constraints 
        C_total = np.concatenate([C_lateral, C_vertical], axis=0)
        lb_total = np.concatenate([lg_lateral, lg_vertical], axis=0)
        ub_total = np.concatenate([ug_lateral, ug_vertical], axis=0)                   
        acados_solver.constraints_set(mpc_time_idx, 'C', C_total, api='new')
        acados_solver.constraints_set(mpc_time_idx, 'lg', lb_total)
        acados_solver.constraints_set(mpc_time_idx, 'ug', ub_total)
    # terminal constraints
    # x_ref_terminal = x_ref_mpc[traj_time_idx+N_mpc]
    # solver.ocp.constraints.idxbx_e = np.array(range(nx))
    # solver.ocp.constraints.lbx_e = x_ref_k 
    # solver.ocp.constraints.ubx_e = x_ref_k     
    # update terminal tracking cost
    acados_solver.cost_set(N_mpc,'yref', x_ref_k)
    # warm-start the terminal node 
    if traj_time_idx == 0:
        acados_solver.set(N_mpc, 'x', x_ref_k)
    else:
        acados_solver.set(N_mpc, 'x', x_warm_start_k)    
    # solve OCP
    if solver.ocp.solver_options.nlp_solver_type == 'SQP_RTI':
        # QP preparation rti_phase:
        print('starting RTI preparation phase ' + '...')
        acados_solver.options_set('rti_phase', 1)
        t_prep = time.time()
        status = acados_solver.solve()
        elapsed_prep = time.time() - t_prep
        print('RTI preparation phase took ' + str(elapsed_prep) + " seconds")
        # feedback rti_phase
        print('starting RTI feedback phase ' + '...')
        acados_solver.options_set('rti_phase', 2)
        t_feedback = time.time()
        status = acados_solver.solve()
        elapsed_feedback = time.time() - t_feedback
        print('RTI feedback phase took ' + str(elapsed_feedback) + " seconds")
        acados_solver.print_statistics()
        if status == 0:
            print("HOORAY ! found a solution after :", 
            elapsed_prep+elapsed_feedback, " seconds")
        else:
            raise Exception(f'acados returned status {status}.')
    else:
        t = time.time()
        status = acados_solver.solve()
        elapsed_time= time.time() - t
        acados_solver.print_statistics()
        if status == 0:
            print("HOORAY ! found a solution after :", 
            elapsed_time, " seconds")
        else:
            raise Exception(f'acados returned status {status}.')
    # save open-loop trajectories
    x_sol = np.array([acados_solver.get(i,"x") for i in range(N_mpc+1)])
    u_sol = np.array([acados_solver.get(i,"u") for i in range(N_mpc)])
    solver.X_sim[traj_time_idx] = x_sol
    solver.U_sim[traj_time_idx] = u_sol
    # get desired generalized position 
    q_des = np.concatenate(
        [
         x_sol[0, 9:12], 
         np.array(
             model.casadi_model.q_plus(x_ref_N[0][12:16], x_sol[0, 12:15]*dt)
         ).squeeze(),
         x_sol[0, 15:27]
        ]
    )
    q_des_next = np.concatenate(
        [
         x_sol[1, 9:12], 
         np.array(
             model.casadi_model.q_plus(x_ref_N[1][12:16], x_sol[1, 12:15]*dt)
         ).squeeze(),
         x_sol[1, 15:27]
        ]
    )
    # get desired generalized velocity
    v_des = x_sol[0, 27:]
    v_des_next = x_sol[1, 27:]
    # get desired contact pose in the local contact frame
    LOCAL_R_FR = ee_fk[0](q=q_des)['ee_rot'].T
    LOCAL_R_FL = ee_fk[1](q=q_des)['ee_rot'].T
    LOCAL_R_HR = ee_fk[2](q=q_des)['ee_rot'].T
    LOCAL_R_HL = ee_fk[3](q=q_des)['ee_rot'].T
    # compute J.T @ f
    Jtf = ee_jacobians[1](q=q_des)['J'][:3].T @ (LOCAL_R_FL @ u_sol[0,:3]) +\
           ee_jacobians[0](q=q_des)['J'][:3].T @ (LOCAL_R_FR @ u_sol[0, 3:6]) + \
           ee_jacobians[3](q=q_des)['J'][:3].T @ (LOCAL_R_HL @ u_sol[0, 6:9]) + \
           ee_jacobians[2](q=q_des)['J'][:3].T @ (LOCAL_R_HR @ u_sol[0, 9:12])
    Jtf_next = ee_jacobians[1](q=q_des_next)['J'][:3].T @ (LOCAL_R_FL @ u_sol[1,:3]) +\
           ee_jacobians[0](q=q_des_next)['J'][:3].T @ (LOCAL_R_FR @ u_sol[1, 3:6]) + \
           ee_jacobians[3](q=q_des_next)['J'][:3].T @ (LOCAL_R_HL @ u_sol[1, 6:9]) + \
           ee_jacobians[2](q=q_des_next)['J'][:3].T @ (LOCAL_R_HR @ u_sol[1, 9:12])
    # compute inverse dynamics 
    tau_ff = np.array(model.casadi_model.rnea(q_des, v_des, u_sol[0, 12:]) - Jtf).squeeze()
    tau_ff_next = np.array(model.casadi_model.rnea(q_des_next, v_des_next, u_sol[1, 12:]) - Jtf_next).squeeze()
    # interpolate solution for low-level torque control
    q_interpol, qdot_interpol, tau_ff_interpol = interpolate_one_step(
        rmodel, dt, dt_ctrl, 
        q_des, q_des_next, 
        v_des, v_des_next,
        tau_ff[6:], tau_ff_next[6:]   
    )
    # torque control loop
    for ctrl_time_idx in range(N_interpol):
        q_meas, qdot_meas = simulator.robot.get_state()
        tau_k = \
        Kp @ (q_interpol[ctrl_time_idx, 7:] - q_meas[7:]) + \
        Kd @ (qdot_interpol[ctrl_time_idx, 6:] -  qdot_meas[6:])
        simulator.robot.send_joint_command(tau_k)
        # step simulation 
        simulator.env.step(sleep=True)  
    # save closed-loop solution
    X_sol[traj_time_idx+1] = x_sol[0]
    U_sol[traj_time_idx] = u_sol[0]
    # warm-start solver from the previous solution 
    x_warm_start_N = np.concatenate([x_sol[1:], x_sol[-1].reshape(1, nx)]) #x_sol
    u_warm_start_N = np.concatenate([u_sol[1:], u_sol[-1].reshape(1, nu)]) #u_sol
    # update initial condition
    # OL-MPC
    x0 = x_sol[1]
    # CL-MPC